# Demo - Performance Comparison (CIFAR10)

## 1. Load CIAFR10

In [1]:
# https://github.com/RobustBench/robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from utils import l2_distance

images, labels = load_cifar10(n_examples=50)
device = "cuda"

Files already downloaded and verified


## 2. Standard Accuracy

In [2]:
model_list = ['Wong2020Fast', 'Rice2020Overfitting', 'Carmon2019Unlabeled']
for model_name in model_list:
    model = load_model(model_name, norm='Linf').to(device)
    acc = clean_accuracy(model, images.to(device), labels.to(device))
    print('Model: {}'.format(model_name))
    print('- Standard Acc: {}'.format(acc))

Model: Wong2020Fast
- Standard Acc: 0.92
Model: Rice2020Overfitting
- Standard Acc: 0.86
Model: Carmon2019Unlabeled
- Standard Acc: 0.92


## 3. Torchattacks, Foolbox and ART

In [3]:
import datetime
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

import torch
import torch.nn as nn
import torch.optim as optim

# https://github.com/Harry24k/adversarial-attacks-pytorch
import torchattacks
print("torchattacks %s"%(torchattacks.__version__))

# https://github.com/bethgelab/foolbox
import foolbox as fb
print("foolbox %s"%(fb.__version__))

# https://github.com/IBM/adversarial-robustness-toolbox
import art
import art.attacks.evasion as evasion
from art.classifiers import PyTorchClassifier
print("art %s"%(art.__version__))

torchattacks 2.13.0
foolbox 3.0.0
art 1.2.0


## 3.1. Linf

### FGSM

In [4]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.FGSM(model, eps=8/255)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfFastGradientAttack(random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.FastGradientMethod(norm=np.inf, batch_size=50,
                                     classifier=classifier, eps=8/255)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.48 (21 ms)
- Foolbox
- Robust Acc: 0.48 (20 ms)
- ART
- Robust Acc: 0.48 (67 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.62 (91 ms)
- Foolbox
- Robust Acc: 0.62 (55 ms)
- ART
- Robust Acc: 0.62 (747 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.68 (16 ms)
- Foolbox
- Robust Acc: 0.68 (26 ms)
- ART
- Robust Acc: 0.68 (221 ms)



### BIM

In [5]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=10)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfBasicIterativeAttack(abs_stepsize=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.BasicIterativeMethod(batch_size=50,
                                       classifier=classifier, eps=8/255,
                                       eps_step=2/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.44 (179 ms)
- Foolbox
- Robust Acc: 0.44 (241 ms)
- ART
- Robust Acc: 0.44 (483 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.58 (2858 ms)
- Foolbox
- Robust Acc: 0.58 (3393 ms)
- ART
- Robust Acc: 0.58 (4762 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.58 (759 ms)
- Foolbox
- Robust Acc: 0.58 (923 ms)
- ART
- Robust Acc: 0.58 (1523 ms)



### PGD

In [6]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfPGD(abs_stepsize=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           classifier=classifier, eps=8/255,
                                           eps_step=2/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.44 (164 ms)
- Foolbox
- Robust Acc: 0.44 (246 ms)
- ART
- Robust Acc: 0.44 (517 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.58 (2883 ms)
- Foolbox
- Robust Acc: 0.58 (3412 ms)
- ART
- Robust Acc: 0.58 (4776 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.58 (687 ms)
- Foolbox
- Robust Acc: 0.58 (923 ms)
- ART
- Robust Acc: 0.58 (1421 ms)



## 3.2. L2

### CW

In [7]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.CW(model, c=1, kappa=0, steps=100, lr=0.01)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=1, initial_const=1,
                                           confidence=0, steps=100, stepsize=0.01)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda'), labels.to('cuda'), epsilons=1)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))

    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.CarliniL2Method(batch_size=50, classifier=classifier, 
                                  binary_search_steps=1, initial_const=1,
                                  confidence=0, max_iter=100,
                                  learning_rate=0.01)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.14 / L2: 0.61 (4399 ms)
- Foolbox
- Robust Acc: 0.32 / L2: 0.41 (4582 ms)
- ART
- Robust Acc: 0.24 / L2: 0.67 (72256 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.22 / L2: 0.56 (43914 ms)
- Foolbox
- Robust Acc: 0.34 / L2: 0.43 (44350 ms)
- ART
- Robust Acc: 0.26 / L2: 0.65 (700187 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.26 / L2: 0.48 (13089 ms)
- Foolbox
- Robust Acc: 0.32 / L2: 0.42 (13199 ms)
- ART
- Robust Acc: 0.26 / L2: 0.62 (205809 ms)



### PGD L2

In [8]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').cuda()
    
    print("- Torchattacks")
    atk = torchattacks.PGDL2(model, eps=128/255, alpha=15/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2PGD(abs_stepsize=15/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=128/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           norm = 2, classifier=classifier, eps=128/255,
                                           eps_step=15/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.68 / L2: 0.5 (179 ms)
- Foolbox
- Robust Acc: 0.68 / L2: 0.5 (259 ms)
- ART
- Robust Acc: 0.68 / L2: 0.5 (478 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.7 / L2: 0.5 (2785 ms)
- Foolbox
- Robust Acc: 0.7 / L2: 0.5 (3498 ms)
- ART
- Robust Acc: 0.7 / L2: 0.5 (4785 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.68 / L2: 0.5 (714 ms)
- Foolbox
- Robust Acc: 0.68 / L2: 0.5 (963 ms)
- ART
- Robust Acc: 0.68 / L2: 0.5 (1426 ms)



## 4. Torchattacks and AutoAttack


In [9]:
from torchattacks.attack import Attack
import autoattack

In [10]:
class AutoAttack(Attack):
    def __init__(self, model, eps, norm, seed, version):
        super(AutoAttack, self).__init__("AutoAttack", model)
        self.adversary = autoattack.AutoAttack(self.model, norm=norm, seed=seed,
                                               eps=eps, version=version, verbose=False)
        self._attack_mode = 'only_default'

    def forward(self, images, labels):
        adv_images = self.adversary.run_standard_evaluation(images.cuda(), labels.cuda(),
                                                            bs=images.shape[0])
        return adv_images

In [11]:
model_list = ['Wong2020Fast']
versions = ['standard', 'plus', 'rand']

In [12]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    for version in versions:
        print("- Torchattacks (%s)"%(version))
        atk = torchattacks.AutoAttack(model, norm='Linf', eps=8/255, seed=5, version=version)
        start = datetime.datetime.now()
        adv_images = atk(images, labels)
        end = datetime.datetime.now()
        acc = clean_accuracy(model, adv_images, labels)
        print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))

        print("- AutoAttack (%s)"%(version))
        atk = AutoAttack(model, norm='Linf', eps=8/255, seed=5, version=version)
        start = datetime.datetime.now()
        adv_images = atk(images, labels)
        end = datetime.datetime.now()
        acc = clean_accuracy(model, adv_images, labels)
        print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks (standard)
- Robust Acc: 0.4 (82762 ms)
- AutoAttack (standard)
- Robust Acc: 0.4 (107698 ms)
- Torchattacks (plus)
- Robust Acc: 0.4 (175907 ms)
- AutoAttack (plus)
- Robust Acc: 0.4 (199065 ms)
- Torchattacks (rand)
- Robust Acc: 0.44 (93533 ms)
- AutoAttack (rand)
- Robust Acc: 0.44 (93569 ms)

